In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial
from copy import copy

import numpy as np
from pprint import pprint

In [2]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        


### Adapted functions
get_precedent() is an adaptation of get_cells()
find_types()  is an adaptation of find_cells()

In [3]:
def get_precedent(evaluations, options=None):
    """Return code components that contribute to the creation of a given evaluation"""
    result = copy(evaluations)
    querier = DependencyQuerier(options)
    visited = set()
    code_component_op = {}
    operations = set()
    while result:
        nodes_to_visit, visited, found = querier.navigate_dependencies(result, visited=visited)
        result = []
        component_to_visit = []
        new_ops = set()
        # Extract operations from visited evaluations
        for context in visited:
            evaluation = context.evaluation

            # Evaluation operations
            component = evaluation.code_component
            operation = find_types(component, code_component_op)
            if operation and operation not in operations:
                new_ops.add(operation)
                operations.add(operation)
        
        # Expand dependencies to all evaluations of new operations
        for operation in new_ops:
            if operation.this_block:
                result.extend(operation.this_block.recursive_evaluations())

    return operations

In [50]:
def find_types(code_component, found=None):
    """Find specific component typesfrom code_components"""
    found = found or {}
    if code_component in found:
        pass
    elif code_component.type:
        found[code_component] = code_component
    elif not code_component.container_id:
        found[code_component] = None
    else:
        found[code_component] = find_types(code_component.container.this_component, found=found)
    return found[code_component]

### Test case

Here starts my (only) test case. The flow goes through cells defining module, mae and rmse, variable definitions, variable manipulation and _var_final_ being our target variable, being dependent of its previous cells.


In [5]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean(np.abs((x - y)))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

def stress_func(m, n):
    temp = m * n
    x = 10
    y = 100
    
    temp = np.random.rand()*temp
    
    final = np.max([int(x+y), int(temp)])
    
    return final

In [6]:
var_a = 10

In [7]:
var_b = -100

In [8]:
x = 50
y = np.max([int(x+y), int(temp)])

In [9]:
rme_err = mae(var_a, var_b)

In [10]:
var_c = stress_func(x, y)

In [11]:
var_d = var_c * 234
var_c = var_d / 3

In [12]:
var_e = mae(var_c, y)
x = var_e + 10

In [13]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, y))

Evaluation(id=141, checkpoint=24.007674867000002, code_component_id=871, activation_id=134, repr=110.0)


### New functions test

In [51]:
evals = get_evaluation(141)
code_comp_list = get_precedent([evals])

In [53]:
code_comp_list

{code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 127, '<class ''type''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 345, '<class ''function''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 346, '<class ''int''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 652, 'mae', 'function_def', 'w', 7, 0, 8, 35, 616).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 660, 'x', 'param', 'w', 7, 8, 7, 9, 652).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 661, 'y', 'param', 'w', 7, 11, 7, 12, 652).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 664, 'np.mean(np.abs((x - y)))', 'call', 'r', 8, 11, 8, 35, 652).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 667, 'np.mean', 'attribute', 'r', 8, 11, 8, 18, 652).,
 code_component(5e0867af-9225-48b6-9b2e-1d2608daf153, 669, 'np', 'name', 'r', 8, 11, 8, 13, 652).,
 code_component(5e08

In [49]:
a = []
for i in code_comp_list:
    #a[i.id] = [i.name, i.type, i.mode, i.first_char_line, i.last_char_line]
    a.append(i.to_dict())
    
a

[OrderedDict([('trial_id', '5e0867af-9225-48b6-9b2e-1d2608daf153'),
              ('id', 875),
              ('name', 'rmse'),
              ('type', 'name'),
              ('mode', 'r'),
              ('first_char_line', 2),
              ('first_char_column', 38),
              ('last_char_line', 2),
              ('last_char_column', 42),
              ('container_id', 861)]),
 OrderedDict([('trial_id', '5e0867af-9225-48b6-9b2e-1d2608daf153'),
              ('id', 738),
              ('name', 'temp'),
              ('type', 'name'),
              ('mode', 'w'),
              ('first_char_line', 18),
              ('first_char_column', 4),
              ('last_char_line', 18),
              ('last_char_column', 8),
              ('container_id', 711)]),
 OrderedDict([('trial_id', '5e0867af-9225-48b6-9b2e-1d2608daf153'),
              ('id', 847),
              ('name', 'mae(var_c, var_b)'),
              ('type', 'call'),
              ('mode', 'r'),
              ('first_char_line',

In [43]:
fun_names = [d['name'] for d in a if d.get('type') == 'name']

In [46]:
filtered_list = [d for d in a if d.get('name') not in fun_names]

In [47]:
filtered_list

[OrderedDict([('trial_id', '5e0867af-9225-48b6-9b2e-1d2608daf153'),
              ('id', 847),
              ('name', 'mae(var_c, var_b)'),
              ('type', 'call'),
              ('mode', 'r'),
              ('first_char_line', 1),
              ('first_char_column', 8),
              ('last_char_line', 1),
              ('last_char_column', 25),
              ('container_id', 843)]),
 OrderedDict([('trial_id', '5e0867af-9225-48b6-9b2e-1d2608daf153'),
              ('id', 753),
              ('name', 'np.max([int(x+y), int(temp)])'),
              ('type', 'call'),
              ('mode', 'r'),
              ('first_char_line', 20),
              ('first_char_column', 12),
              ('last_char_line', 20),
              ('last_char_column', 41),
              ('container_id', 711)]),
 OrderedDict([('trial_id', '5e0867af-9225-48b6-9b2e-1d2608daf153'),
              ('id', 692),
              ('name', 'np.sqrt(np.mean((x - y)**2))'),
              ('type', 'call'),
            

### Outputing the dependencies of a variable

print_precedents() trying to output a cleaner output from get_precedent() results

In [15]:
def print_precedents(evals):
    code_componenents = get_precedent([evals])
    
    op_dict = {}
    for i in reversed(list(code_comp_list)):
        code_comp = i
        obj_eval = Evaluation((i.trial_id, i.id))
        obj_act = Activation((obj_eval.trial_id, obj_eval.activation_id))

        if  ((code_comp.type == 'call') and (code_comp.mode == 'r')) or \
            (code_comp.type == 'name' and code_comp.mode == 'w'): 
                op_dict[round(float(obj_eval.checkpoint), 6)] = [round(float(obj_act.start_checkpoint),4), \
                round(float(obj_eval.checkpoint), 4), code_comp.name, code_comp.type, code_comp.mode]  
    
    return op_dict

In [16]:
print_precedents(evals)

{15.716617: [15.7165, 15.7166, 'var_b', 'name', 'w'],
 15.719194: [15.719, 15.7192, 'mae(var_c, var_b)', 'call', 'r'],
 15.714099: [14.9807, 15.7141, 'np.abs((x - y))', 'call', 'r'],
 15.718967: [15.719, 15.719, 'var_c', 'name', 'w'],
 15.714625: [15.7141, 15.7146, 'np.sqrt(np.mean((x - y)**2))', 'call', 'r'],
 15.717357: [14.9807, 15.7174, 'var_c', 'name', 'w'],
 15.719159: [15.719, 15.7192, 'var_e', 'name', 'w'],
 15.7188: [14.9807, 15.7188, 'var_d', 'name', 'w'],
 15.712702: [14.9807, 15.7127, 'np.mean(np.abs((x - y)))', 'call', 'r'],
 15.719672: [14.9807, 15.7197, 'rmse(var_c, var_e)', 'call', 'r'],
 15.714862: [14.9807, 15.7149, 'np.mean((x - y)**2)', 'call', 'r']}

In [23]:
import pandas as pd

pd.DataFrame(my_dict, columns=['a'])

ValueError: If using all scalar values, you must pass an index